In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import random
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from datetime import timedelta
import seaborn as sns

In [2]:
def pd_dttm_import(want_cols):
    '''Parameters: takes in list of column names wanted that will be
    used in pandas .read_csv method
    
    Results: Outputs the indices of the columns names that are date
    values and uses resulting list of indices as parse_dates parameter
    in pandas .read_csv method'''
    dt_ind = []
    for i in want_cols:
        if 'dttm' in i.lower() or 'date' in i.lower():
            dt_ind.append(want_cols.index(i))
    return dt_ind

def to_time_delta_by_col(df, col, time2, time1):
    '''create new copy of data frame taking only not null values - to be used when wanting times matched up with
    single other column
    '''
    #create new copy of data frame taking only not null values
    df_ind = df[df[time1].notna() & df[time2].notna()]
    frame = pd.DataFrame(df_ind[col])
    frame['time_delta'] = (df_ind[time2] - df_ind[time1]).apply(lambda x: timedelta.total_seconds(x))
    return frame

def to_time_delta(df, time2, time1):
    '''Return series of difference between two datetime columns
    '''
    #create new copy of data frame taking only not null values
    return (df[time2] - df[time1]).apply(lambda x: timedelta.total_seconds(x) if not isinstance(x, pd._libs.tslibs.nattype.NaTType) else None)

In [3]:
col_of_interest = ['Call Number', 'Unit ID', 'Call Type',
       'Call Date', 'Received DtTm', 'Dispatch DtTm', 'Response DtTm', 'On Scene DtTm', 'Transport DtTm',
       'Hospital DtTm', 'Call Final Disposition', 'Available DtTm', 'Battalion', 
       'Final Priority', 'ALS Unit',
       'Call Type Group', 'Unit Type',
       'Neighborhooods - Analysis Boundaries']

Want only alarms and medical - NEED percentage of total that is that

-they are of most interest in terms of what i think the homeless population would increase calls for

-want to eliminate call types that have lots of different units respond (fires) as that skews the response time

In [ ]:
'''
call_to_disp = df['Dispatch DtTm']-df['Received DtTm']
disp_to_resp = df['Response DtTm']-df['Dispatch DtTm']
resp_to_scene = df['On Scene DtTm'] - df['Response DtTm']
total_time = df['On Scene DtTm'] - df['Received DtTm']'''

In [4]:
df = pd.read_csv('EMS_data.csv', usecols=col_of_interest, parse_dates=pd_dttm_import(col_of_interest))
print(f'Number of entries: {len(df)}')

/home/christopher/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Number of entries: 5324768


In [8]:
#change data type of neighborhoods column
df['Neighborhooods - Analysis Boundaries'] = df['Neighborhooods - Analysis Boundaries'].astype(str)
#drop rows where any of the of-interest time data is NaN
#comfortable dropping this because if any is missing it suggests that the call led to no interaction with EMS
for i in ['Received DtTm', 'Dispatch DtTm', 'Response DtTm', 'On Scene DtTm']:
    print(f'Number of entries where {i} is NaN: {len(df[df[i].isnull()])}')
df.dropna(axis = 0, how = 'any', subset = ['Received DtTm', 'Dispatch DtTm', 'Response DtTm', 'On Scene DtTm'], inplace=True)
#check percentage of remaining data that has neighborhood value of 'None' and drop those rows - validate decision based on percentage
a = len(df[df['Neighborhooods - Analysis Boundaries'] == 'None'])/len(df)
print(f'Proportion of data where neigh is None: {a}')
df = df[df['Neighborhooods - Analysis Boundaries'] != 'None']

Number of entries where Received DtTm is NaN: 0
Number of entries where Dispatch DtTm is NaN: 0
Number of entries where Response DtTm is NaN: 417166
Number of entries where On Scene DtTm is NaN: 1201775
Proportion of data where neigh is None: 0.0013464225065687055


In [9]:
df['call_to_disp'] = to_time_delta(df,'Dispatch DtTm', 'Received DtTm')
df['disp_to_resp'] = to_time_delta(df, 'Response DtTm','Dispatch DtTm')
df['resp_to_scene'] = to_time_delta(df, 'On Scene DtTm','Response DtTm')
df['time_to_hosp'] = to_time_delta(df, 'Hospital DtTm', 'Transport DtTm')

In [10]:
#drop original DtTm data
df.drop(['Received DtTm', 'Dispatch DtTm', 'Response DtTm', 'On Scene DtTm', 'Transport DtTm',
       'Hospital DtTm'], axis = 1, inplace = True)

In [11]:
df.to_csv('Time_Delta_Data', index=False)